In [2]:
import keras
keras.__version__

'2.1.5'

### convnet 개요
- 컨볼류션 신경망 (convolutional neural network)를 소개하겠다.
- cnn을 사용하여 이미지를 분류하는 문제를 해결하고 특히 학습할 이미지의 수가 적을 때를 다루겠다 (현실적으로 이런 경우가 많다)
- convnet의 개념을 설명하고 이를 이용하여 MNIST 분류 문제를 다룬다
- 앞의 간단한 코드예에서는 MNIST 인식률이 97.8%의 성능을 보였다.
- 아래의 코드에서 Conv2D와 MaxPoolinf2D 두 계층을 사용하여 신경망을 구성한다
- 입력 텐서로 (image_height, image_width, image_channels)구조를 사용한다 (배치의 크기는 별도로 지정한다)
- 여기서는 (28, 28, 1)의 크기를 갖으며 이를 input_shape인자로 사용하면 된다

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

- 위에서 정의한 covnet의 구조를 보겠다

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


- conv2d와 MaxPooling 계층의 출력이 3차원 텐서로서 구조가 (height, width, channels)로 된다
- 폭과 높의 값은 네트워크가 깊어지면서 점차 작아진다.
- 채널 크기는 conv2d 계층을 만들 때 정해준다 (위에서 32나 64)
- 출력단의 (3, 3, 64) 구조의 출력 텐서를 이제 Dense 구조의 최종 분류 네트워크로 보내야 한다.
- 분류기는 벡터, 즉 1차원 텐서를 사용하므로, 현재 가지고 있는 3D 텐서를 먼저 1차원으로 만들어야 한다(flatten)

In [5]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

- 10개 숫자를 구분해야 하므로 10개의 출력을 갖는 softmax 활성화 함수를 사용한다

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

- (3, 3, 64) 구조의 출력을 (576,) 구조로 평탄화 한 후 두개의 Dense 계층을 통과시켰다.
- 이제 훈련을 시키면 된다.

In [7]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 24s 402us/step - loss: 0.1749 - acc: 0.9452
Epoch 2/5
60000/60000 [==============================] - 25s 419us/step - loss: 0.0512 - acc: 0.9839
Epoch 3/5
60000/60000 [==============================] - 25s 423us/step - loss: 0.0354 - acc: 0.9891
Epoch 4/5
60000/60000 [==============================] - 26s 438us/step - loss: 0.0280 - acc: 0.9919
Epoch 5/5
60000/60000 [==============================] - 26s 431us/step - loss: 0.0221 - acc: 0.9933


- 이제 테스트 데이터로 모델의 성능을 검증한다

In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 137us/step


In [10]:
test_acc

0.9909

- 앞에서 소개한 Dense 형태의 '단순한' 구조의 신경망 모델에 비해 convnet은 성능이 97.8%에서 99.1%로 크게 개선된 것을 알 수 있다. 그 이유가 무엇일까? 이를 이해하기 위해서는 Conv2D 계층과 MaxPooling 계층의 동작을 먼저  이해해야 한다.
### 컨볼류션 동작
- Dense 구조의 특징은 입력 이미지 전체 픽셀의 특성을 한번에 학습하는 것이라면, 컨볼류션은 입력 신호의 지역적인 특성들을 학습한다는 것이 가장 큰 차이점이다. 위 코드에서는 이 "지역"의 크기를 3x3 픽셀 크기로 나누어서 학습을 하였다. 즉, 각 지역의 패턴에 들어 있는 특징 (edge, texture)등을 찾아내는 것이다. 아래 그림에 지역의 패턴을 찾는 예를 보였다.

<img src="https://www.dropbox.com/s/c9lk7j3zh582wsx/cnn-1.png?raw=1", width=200>

- 이러한 성격으로인해 cnn은 다음과 같은 중요한 두가지 특성을 갖는다. 
 - 마이크로 패턴의 위치 이동에 무관하게 동작한다: 특정 패턴이 이미지 상에서 나타나는 위치가 변경되어도 이를 찾아낼 수 있다. 따라서 이미지를 분석하는데 매우 유용하다.
 - 패턴의 공간적인 계층구조(hierarchy)를 찾아낼 수 있다: 계층을 지나면서 점차 추상적인 내용을 파악할 수 있다. 저층에서는 기본적인 패턴을 찾고, 다음 계층에서는 이를 이용한 보다 복잡한 패턴을 찾는 것이다. 즉, 세밀한 것들이 모여서 큰 그림을 만들고 사람들이 이해하는 개념을 만들어 낼 수 있다.
 
 <img src=" https://www.dropbox.com/s/hq0wrdga4cgpny3/cnn-2.png?raw=1", width=400>


- 컨볼류션은 3D를 사용하여 이루어지는데 (이를 피처 맵이라고 부른다), 평면을 구성하는 2차원 축 데이터와(폭, 높이), 깊이(depth) 축으로 구성된다(이를 채널이라고도 부른다).
- RGB 신호로 코딩된 경우, 깊이 축의 차원은 세가지 색을 각각 나타내는 3이 된다. 흑백으로 코딩된 경우 (MNIST처럼) 깊이는 1이 된다.
- 컨볼류션 동작은 입력 피처맵으로부터 일정 영역을 추출하며 이들 영역들에 대해서 모두 동일한 변환(필터링)을 수행한다. 그 결과로 출력 피처 맵을 생성한다. 이 출력 피처맵도 역시 3D 텐서 구조를 갖는다. 계층을 통과할 때 깊이의 크기를 변경할 수 있다.
- 여기서 주의할 것은 출력 피처맵 깊이의 각 채널이 여전히 RGB 신호 정보를 유지하는 것이 아니라는 것이다. 이 채널은 입력 신호를 다양한 필터를 사용하여 필터링한 새로운 신호들의 집합을 나타낸다. 

- 앞의 MNIST 예에서, 첫번째 컨볼류션에서 입력 피처맵의 크기는 (28, 28, 1)이고 출력 피처맵은 (26, 26, 32)였다. 32 종류의 필터를 사용하였으므로 출력의 깊이가 32개가 되었다. 32개의 출력 채널은 각각 26x26크기의 값으로 구성되는데, 이는 입력에 대한 각 필터의 출력 반응(response) 맵이 된다. 이것이 바로 피처맵이라는 이름이 붙은 이유이다.
- 깊이 축에서 각 디멘션이 피처를 나타내는 것이고, 각 피처에 대한 2차원 텐서는 공간상의 맵을 나타낸다.
- 아래 그림에 반응 맵을 타나냈다.

<img src="https://www.dropbox.com/s/elnli0sy2vwt56r/cnn-3.png?raw=1", width=400>

- 컨볼류션은 다음과 같이 두가지 주요 파라미터로 정의된다.
 - 입력에서 추축하는 패치의 크기: 보통 3x3, 5x5등을 사용한다.
 - 출력 피처맵의 깊이: 필터의 수를 말하며, 위의 예에서는 두개의 계층에서 각각 32로 시작하고 64를 사용했다.
- 컨볼류션 수행은 이 윈도우를 옆으로 슬라이딩 하면서 필터링 작업을 반복한다. 필터 가중치 매트릭스를 컨볼류션  커널이라고 한다. 필터 출력은 깊이 크기를 갖는 1차원 벡터가 된다.
- 이 동작을 아래에 나타냈다.

<img src="https://www.dropbox.com/s/qe5g7nmrgnojg11/cnn-4.png?raw=1", width =600 >
 

- 출력의 폭과 높이는 입력과 다를 수 있는데, 그 이유는 다음과 같다.
 - 경계에 데이터 부족해서 입력 수만큼 만들지 못하는 경계 효과를 말하며 이를 보정하기 위해서 패딩을 할 수 있다.
 - 스트라이드를 적용하면 크기가 줄어들 수 있다.
 
- 경계효과란 아래 그림처럼 5x5 타일이 3x3으로 객수가 줄어드는 것을 말한다. (3x3 윈도우를 사용하는 경우). 앞의 예에서는 28x28이 26x26으로 2 만큼 줄어들었다.

<img src="https://www.dropbox.com/s/3ndmrv9kku6n6gk/cnn-5.png?raw=1", width=500>

- 출력 피처맵의 크기를 입력단과 같게 만들려면 패딩을 해주어야 한다. 예를 5x5 입력에 대해 3x3 윈도우를 사용하면서 출력의 크기를 5x5로 같게 유지하려면 아래 그림과 같이 상하좌우에 1줄씩 패딩을 해주어야 한다.

<img src="https://www.dropbox.com/s/6ghx7zatridmslb/cnn-6.png?raw=1", width=500>

- Conv2D 계층에서는 padding인자를 사용하여 패딩을 지정할 수 있다. valid로 설정하면 패딩을 사용하지 말라는 뜻이다. 즉, 유효한 데이터만 사용하라는 뜻이고, same을 지정하면 출력의 차원이 입력과 같아지도록 적절한 수의 패딩을 자동으로 입력하라는 것이다.
- strides를 사용하여 출력의 수를 바꿀 수 있는데, strides를 1로 하면 한 칸씩 옆으로 이동하면서 패치를 추출하고 strides가 2면 2칸씩 건너 이동하면서 패치를 추출한다. 아래 그림은 strides가 2인 경우를 나타냈고 출력이 2x2가 되는 것을 보였다.

<img src="https://www.dropbox.com/s/sebpa4bfdvgyxk7/cnn-7.png?raw=1", width=500>

- strides가 2면 약 1/2로 다운샘플링 된다. 경계효과로 인해 약간의 차이는 있지만 데이터 차원이 크면 거의 1/2로 된다. 실제로 스트라이드는 잘 사용되지 않는다. 피처맵의 크기를 줄이는 것이 목적이라면 strides를 사용하는 것보다, 다음에 설명할 max-sampling 방법을 사용하면 된다.
- 맥스 풀링: MaxPooling2D 계층을 사용한다. 이는 샘플의 수를 줄이는 효과가 있는데 위의 예에서는 26x26 크기가 13x13으로 줄어들었다. 보통 2x2 윈도우 내에서 최대값을 택하는 방법을 사용하며 strides는 2를 택한다. 반면에 컨볼류션은 보통 3x3 크기의 윈도우에 대해서 실행하며 strides는 1을 택한다.

- 이렇게 맥스 풀링을 하여 데이터를 다운샘플링하는 이유는 다음과 같다
 - 컨볼류션만 연속 수행하면 입력 이미지의 계층구조 정보를 찾아내기가 어렵다. 입력 신호 전체의 성격을 추출하는 기능이 필요하다
 - 컨볼류션만 계속하면 학습시켜야 할 파라미터 수가 급격히 늘어나서 학습 속도가 느려지고, 과적합할 가능성이 높아진다.

- 즉, 다운샘플링이 필요한 이유는 계층을 지나가면서 그림 전체의 구조를 파악할 수 있는 기회를 만들고, 파라미터 수를 적절히 줄이기 위해서이다.
- 맥스 풀릴을 하는 것과 유사한 효과로, 패치를 선택할때  strides 수행할 수도 있다. 또는 맥스풀링 대신 패치내의 최대치가 아니라 평균치를 택하는 방법도 있다. 그러나 이러한 두가지 방법보다 맥스풀링의 성능이 가장 았다.
- 이를 달리 표현하면 여러 피처의 평균적인 존재값(average presence)보다, 최대 값(maximal presence)을 추출하는 것이 좋은 성능을 보인 것이다.

- cnn의 핵심은 세가지로 구성된다. 피처맵, 컨볼류션, 맥스 풀링.
